# Классификация изображений. Сверточные сети. Полносвязные сети с предобучением и дообучением.

Выполнили Вилюмсон Александр, Николаев Егор из группы 4136 

In [1]:
import os

base_dir = "archive/flowers"
train_dir = os.path.join(base_dir, "train")
val_dir = os.path.join(base_dir, "val")

def count_images_per_class(directory):
    counts = {}
    if os.path.exists(directory):
        for cat in os.listdir(directory):
            category_path = os.path.join(directory, cat)
            if os.path.isdir(category_path):
                counts[cat] = len([img for img in os.listdir(category_path) if img.endswith(".jpg")])
    return counts

train_counts = count_images_per_class(train_dir)
val_counts = count_images_per_class(val_dir)

# Вывод результата
print("Количество изображений в выборке 'train':")
for category, count in train_counts.items():
    print(f"  {category}: {count}")

print("\nКоличество изображений в выборке 'val':")
for category, count in val_counts.items():
    print(f"  {category}: {count}")

Количество изображений в выборке 'train':
  daisy: 611
  dandelion: 841
  rose: 627
  sunflower: 586
  tulip: 787

Количество изображений в выборке 'val':
  daisy: 153
  dandelion: 211
  rose: 157
  sunflower: 147
  tulip: 197
